## We will use this workflow to investigate catagorical supervised machine learning for our behavioral annotations.
## The goal is to achieve maximal classification in each decision category. Any means may be used (we care about predictive accuracy). 

In [36]:
# Required Files

# 'exp_block_RM160190920S3'
# 'kin_block_RM160190920S3'


In [37]:
import pickle
from Analysis_Utils import preprocessing_df as preprocessing
from Analysis_Utils import query_df
import DataStream_Vis_Utils as utils
import Classification_Utils as CU
import pandas as pd
import pdb
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from Classification_Visualization import visualize_models
import numpy as np
import h5py
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, SGDClassifier
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Ignore ipython warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib qt
%matplotlib inline

In [38]:
# 1. 

# load saved block pickles
exp_block_df = pd.read_pickle('exp_block_RM160190920S3')
kin_block_df = pd.read_pickle('kin_block_RM160190920S3')


# RENAME variables
et = 0
el = 0
wv = 5
window_length = 4
pre = 2

# trailize kin and exp data
hot_vector, tt, feats, e \
            = CU.make_s_f_trial_arrays_from_block(kin_block_df, exp_block_df, et, el, wv, window_length, pre)

# convert exp block to df
exp_block_df = CU.import_experiment_features_to_df(e) # EXP DF

# convert trialized kin block to df
num_labeled_trials = tt.shape[1]
kin_block_df_arm = CU.split_kin_trial_to_df(tt, num_labeled_trials, 0) # KIN XYZ DF
kin_block_df_prob = CU.split_kin_trial_to_df(tt, num_labeled_trials, 1)# KIN PROB DF

In [39]:
display(exp_block_df)


,Robot Velocity X,Robot Velocity Y,Robot Velocity Z,unused idx 4,unused idx 5,unused idx 6,Reward Zone,Robot Position X,Robot Position Y,Robot Position Z,Licking,Moving
0,"[19.57800006866455, 19.586000204086304, 19.592...","[1.134721488406e-311, 1.134721488406e-311, 1.1...","[1.134721488406e-311, 1.134721488406e-311, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.12161461031004099, 0.11989626901422086, 0.1...","[0.13465971849094918, 0.13334339265470496, 0.1...","[0.5819617077139747, 0.5821435836462767, 0.580...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[86.20000004768372, 86.20700001716614, 86.2140...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.1031685979364331, 0.10160562134234435, 0.10...","[0.09446762141204827, 0.09491717997780946, 0.0...","[0.37494187306167304, 0.37504015206763636, 0.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[105.33000016212463, 105.33700013160706, 105.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.1124804839930933, 0.11232491232975404, 0.11...","[0.14327286463893316, 0.14328441892334745, 0.1...","[0.45026945757605336, 0.45001221743368514, 0.4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,"[114.55200004577637, 114.55900001525879, 114.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.091684976564162, 0.09137528549784565, 0.090...","[0.07402640392614919, 0.07364313065067823, 0.0...","[0.342779338823133, 0.3414816539215574, 0.3482...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,"[121.63100004196167, 121.63900017738342, 121.6...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.10916277306073645, 0.10916740357752011, 0.1...","[0.1665741563877634, 0.16718713177898362, 0.16...","[0.5593263651428961, 0.5577495302772866, 0.558...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,"[133.8090000152588, 133.81500005722046, 133.82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.11733207807065045, 0.11764354880062008, 0.1...","[0.1714164577953775, 0.17125379556476536, 0.17...","[0.37244896551811046, 0.3729884279630203, 0.37...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,"[162.71000003814697, 162.71800017356873, 162.7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.08968536688307911, 0.09124727158121332, 0.0...","[0.12149697640025353, 0.12426831139262536, 0.1...","[0.4501490073186823, 0.44961670707833, 0.45139...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7,"[169.73100018501282, 169.739000082016, 169.746...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.12292593624960974, 0.1227751344148591, 0.12...","[0.09758054088077729, 0.09788709029202486, 0.0...","[0.5715529523826407, 0.5698952672788973, 0.573...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8

In [40]:
display(kin_block_df_arm)

# to access trial 0
display(kin_block_df_arm.loc[(0)])

# to access trial 0, X pos values 
display(kin_block_df_arm.loc[(0, "X")])

# to access 'Handle' column values for all trials
display(kin_block_df_arm['Handle'])

Handle  \
0  X  [-0.032626749824766335, -0.032611455491496466,...   
   Y  [-0.04224659359608452, -0.042245619479899035, ...   
   Z  [-0.011773075334736606, -0.01216951505200217, ...   
1  X  [-0.018441636951032084, -0.018438701996585278,...   
   Y  [-0.029859723129974397, -0.029813068413231524,...   
...                                                 ...   
48 Y  [-0.0558441489704188, -0.0558441489704188, -0....   
   Z  [-0.022468464522188147, -0.02225885233092033, ...   
49 X  [-0.009250973595609382, -0.009240513389286413,...   
   Y  [-0.02089749813128419, -0.020889565553759752, ...   
   Z  [-0.027829052304989003, -0.027700341092367586,...   

                                            Back Handle  \
0  X  [-0.017694315636877754, -0.017659000805323958,...   
   Y  [-0.01684424862396891, -0.016857980854288412, ...   
   Z  [-0.017727933321120853, -0.017727933321120853,...   
1  X  [0.03496981458350425, 0.03496981458350425, 0.0...   
   Y  [0.03346674488542277, 0.03117688904682041, 0.0...   
...                                                 ...   
48 Y  [-0.029886122168718793, -0.02896428261599204, ...   
   Z  [-0.035533665189413956, -0.034674347000935773,...   
49 X  [0.03618065130046991, 0.03404015240583993, 0.0...   
   Y  [0.03404662159408792, 0.03207233911627769, 0.0...   
   Z  [0.03393449816305021, 0.03023063243107472, 0.0...   

                                                   Nose  \
0  X  [-0.00358512422854999, -0.0034794068651257323,...   
   Y  [0.005103326426086053, 0.005103326426086053, 0...   
   Z  [-0.004151663309629478, -0.004151663309629478,...   
1  X  [0.0075175255760711835, 0.0075175255760711835,...   
   Y  [0.03526185401631538, 0.03495515364291905, 0.0...   
...                                                 ...   
48 Y  [-0.027331309126481577, -0.026495077852453815,...   
   Z  [-0.028137798151270512, -0.02703011635326167, ...   
49 X  [0.034326164313617666, 0.03002490305417386, 0....   
   Y  [0.03654692128236876, 0.03396209586885449, 0.0...   
   Z  [0.03688931774866559, 0.035811841796966074, 0....   

                                          Left Shoulder  \
0  X  [-0.013543929085432645, -0.0011577413828666584...   
   Y  [0.0018890932131280336, 0.0018890932131280336,...   
   Z  [0.0016781116099069575, 0.001680711223574126, ...   
1  X  [0.03246122433085493, 0.0306168357445287, 0.03...   
   Y  [0.032487005909782485, 0.031019106118592505, 0...   
...                                                 ...   
48 Y  [0.03547106260322202, 0.03543641941637822, 0.0...   
   Z  [0.0004140454252045145, 0.0004084988987227686,...   
49 X  [0.04937252589667919, 0.049593183098101595, 0....   
   Y  [0.046210452300982786, 0.0462952356432006, 0.0...   
   Z  [0.044630793752131775, 0.04447544824296484, 0....   

                                           Left Forearm  \
0  X  [0.0004856357647264938, -6.49332814125378e-05,...   
   Y  [-0.03059706919370302, -0.03084211523744678, -...   
   Z  [0.0021577542244844666, 0.002060769026217548, ...   
1  X  [0.03151376842000518, 0.030772804418860805, 0....   
   Y  [0.032681505667449, 0.032000952421477845, 0.03...   
...                                                 ...   
48 Y  [-0.00029935537857885897, -0.00029935537857885...   
   Z  [0.0027422124610398373, 0.0027555168582002673,...   
49 X  [0.04556745736227295, 0.04512962467940246, 0.0...   
   Y  [0.048197069794576664, 0.04910469861149932, 0....   
   Z  [0.048398011709881596, 0.04893878585968379, 0....   

                                             Left Wrist  \
0  X  [0.019824129095797994, 0.019690540616651195, 0...   
   Y  [0.01380103715785396, 0.014266336991736623, 0....   
   Z  [0.019036866684721532, 0.019036866684721532, 0...   
1  X  [-0.0002990696010311826, -0.000299069601031182...   
   Y  [-0.0025821022481773885, -0.002644646276314150...   
...                                                 ...   
48 Y  [-0.002808033811033973, -0.0033748001745597337...   
   Z  [-0.007783535531095799, -0

,Handle,Back Handle,Nose,Left Shoulder,Left Forearm,Left Wrist,Left Palm,Left Index Base,Left Index Tip,Left Middle Base,...,Right Wrist,Right Palm,Right Index Base,Right Index Tip,Right Middle Base,Right Middle Tip,Right Third Base,Right Third Tip,Right Fourth Finger Base,Right Fourth Finger Tip
X,"[-0.032626749824766335, -0.032611455491496466,...","[-0.017694315636877754, -0.017659000805323958,...","[-0.00358512422854999, -0.0034794068651257323,...","[-0.013543929085432645, -0.0011577413828666584...","[0.0004856357647264938, -6.49332814125378e-05,...","[0.019824129095797994, 0.019690540616651195, 0...","[0.012837949304762404, 0.012917584930398001, 0...","[0.019504144156298753, 0.019495904059472314, 0...","[0.014993859820203547, 0.014993859820203547, 0...","[0.09384601699432049, 0.09384015003768634, 0.0...",...,"[0.002645036074684562, 0.0026525453692605207, ...","[0.016679290744879446, 0.01660171685815603, 0....","[0.0150807544700367, 0.015164399314643898, 0.0...","[0.007397819516587368, 0.007920879786971813, 0...","[0.014583016818042294, 0.014584331099257955, 0...","[0.02540416976679222, 0.026099603922422304, 0....","[0.03216943122356788, 0.016239387949723502, 0....","[0.09124842671088826, 0.09121624895331293, 0.0...","[0.05754187787368732, 0.05899765898730821, 0.0...","[0.1314094244480565, 0.1313906570146837, 0.131..."
Y,"[-0.04224659359608452, -0.042245619479899035, ...","[-0.01684424862396891, -0.016857980854288412, ...","[0.005103326426086053, 0.005103326426086053, 0...","[0.0018890932131280336, 0.0018890932131280336,...","[-0.03059706919370302, -0.03084211523744678, -...","[0.01380103715785396, 0.014266336991736623, 0....","[0.0067752534804801255, 0.006102921308410916, ...","[0.03199159123673115, 0.03259335746679374, 0.0...","[0.011980460471039681, 0.011984871355610317, 0...","[0.10485160150972438, 0.10483291499055776, 0.1...",...,"[0.0018745745278981188, 0.0018729579444367302,...","[0.011558129380417193, 0.011565868137171273, 0...","[0.013610073051118712, 0.013646885004090059, 0...","[0.013840071025534214, 0.014424405736596639, 0...","[0.016525035681669523, 0.01652690647132222, 0....","[0.02744212331243813, 0.027822040309566407, 0....","[0.019220545756543812, 0.01898027292617216, 0....","[0.07749096907819433, 0.10479555708460428, 0.1...","[0.08963220780342743, 0.09082496943324438, 0.0...","[0.12943701049842188, 0.1294342172639193, 0.12..."
Z,"[-0.011773075334736606, -0.01216951505200217, ...","[-0.017727933321120853, -0.017727933321120853,...","[-0.004151663309629478, -0.004151663309629478,...","[0.0016781116099069575, 0.001680711223574126, ...","[0.0021577542244844666, 0.002060769026217548, ...","[0.019036866684721532, 0.019036866684721532, 0...","[0.013935474067963645, 0.014072583425121039, 0...","[0.019227386595052236, 0.019211386995084033, 0...","[0.026767122188823815, 0.026767122188823815, 0...","[0.042574083177901456, 0.04394415689098623, 0....",...,"[-0.02961465093556048, -0.03000406544818068, -...","[0.02667703873168258, 0.027565982628050708, 0....","[0.00985274521700916, 0.00998023146164639, 0.0...","[0.010915218960511753, 0.010936459448741439, 0...","[0.014107217931547627, 0.014092920777550262, 0...","[0.02687630827041471, 0.02748441703127202, 0.0...","[0.14302909990723323, 0.14301283462385422, 0.1...","[0.09026435946153256, 0.0903741782014455, 0.09...","[0.1455296720715347, 0.1455008321685334, 0.145...","[0.15691227319763804, 0.15749973304049011, 0.1..."


Handle                      [-0.032626749824766335, -0.032611455491496466,...
Back Handle                 [-0.017694315636877754, -0.017659000805323958,...
Nose                        [-0.00358512422854999, -0.0034794068651257323,...
Left Shoulder               [-0.013543929085432645, -0.0011577413828666584...
Left Forearm                [0.0004856357647264938, -6.49332814125378e-05,...
Left Wrist                  [0.019824129095797994, 0.019690540616651195, 0...
Left Palm                   [0.012837949304762404, 0.012917584930398001, 0...
Left Index Base             [0.019504144156298753, 0.019495904059472314, 0...
Left Index Tip              [0.014993859820203547, 0.014993859820203547, 0...
Left Middle Base            [0.09384601699432049, 0.09384015003768634, 0.0...
Left Middle Tip             [0.09935298551595026, 0.0992737477219654, 0.09...
Left Third Base             [0.10302734036505228, 0.10305416921094933, 0.1...
Left Third Tip              [0.13731548800630333, 0.119287377382

0   X    [-0.032626749824766335, -0.032611455491496466,...
    Y    [-0.04224659359608452, -0.042245619479899035, ...
    Z    [-0.011773075334736606, -0.01216951505200217, ...
1   X    [-0.018441636951032084, -0.018438701996585278,...
    Y    [-0.029859723129974397, -0.029813068413231524,...
                               ...                        
48  Y    [-0.0558441489704188, -0.0558441489704188, -0....
    Z    [-0.022468464522188147, -0.02225885233092033, ...
49  X    [-0.009250973595609382, -0.009240513389286413,...
    Y    [-0.02089749813128419, -0.020889565553759752, ...
    Z    [-0.027829052304989003, -0.027700341092367586,...
Name: Handle, Length: 150, dtype: object

In [41]:
display(kin_block_df_prob)

Handle  \
0  prob1  [0.03512732432758264, 0.05567533918003194, 0.0...   
   prob2  [0.058885190057698704, 0.05898126582030329, 0....   
   prob3  [-0.013716421424639394, -0.013693936613249781,...   
1  prob1  [0.032752998976757194, 0.030984923351838993, 0...   
   prob2  [0.009502054031549956, 0.009157883525443038, 0...   
...                                                     ...   
48 prob2  [0.0014166557669371314, 0.0011473341081271985,...   
   prob3  [-0.017563905446643786, -0.01731750739883784, ...   
49 prob1  [0.0015552402014028114, 0.04216687383448234, 0...   
   prob2  [0.005511001840391833, 0.004518411310025996, 0...   
   prob3  [0.04144336437631097, 0.039866391077794563, 0....   

                                                Back Handle  \
0  prob1  [-0.016083501458372174, -0.01601266723746722, ...   
   prob2  [-0.014161958413752927, -0.002630186574427852,...   
   prob3  [-0.016620989866167078, -0.01661524985354637, ...   
1  prob1  [0.032355719089109336, 0.030128604161488475, 0...   
   prob2  [0.03209128121018592, 0.03253165718977846, 0.0...   
...                                                     ...   
48 prob2  [-0.04045390558209302, -0.03965125157132149, -...   
   prob3  [-0.03350449421457164, -0.03285186233582894, -...   
49 prob1  [0.0329312662117579, 0.02986006105880625, 0.02...   
   prob2  [0.032864432650598074, 0.02839138274316042, 0....   
   prob3  [0.03410060284239728, 0.03132309164317341, 0.0...   

                                                       Nose  \
0  prob1  [0.02649734767834258, 0.026015159566258245, 0....   
   prob2  [0.0020558710597348494, 0.001537308664303089, ...   
   prob3  [-0.01390886261555324, -0.013895210364783701, ...   
1  prob1  [0.05123365157025589, 0.05143897188617967, 0.0...   
   prob2  [0.03308658250225087, 0.0323216258645373, 0.03...   
...                                                     ...   
48 prob2  [0.024609428654692372, 0.024326931017199575, 0...   
   prob3  [-0.00991476176380538, -0.009886255125910166, ...   
49 prob1  [0.05116596676258673, 0.05160483965208854, 0.0...   
   prob2  [0.049729383507840404, 0.047448815593935864, 0...   
   prob3  [0.04682284565789055, 0.05273617074200884, 0.0...   

                                              Left Shoulder  \
0  prob1  [0.002645036074684562, 0.0026525453692605207, ...   
   prob2  [0.0018745745278981188, 0.0018729579444367302,...   
   prob3  [-0.02961465093556048, -0.03000406544818068, -...   
1  prob1  [0.031918051557689585, 0.030727004669070065, 0...   
   prob2  [0.02987289784849539, 0.02917713974513204, 0.0...   
...                                                     ...   
48 prob2  [0.0017135866551127971, 0.0017213093258805388,...   
   prob3  [-0.0074272679120547845, -0.006731164640330177...   
49 prob1  [0.04569169575963171, 0.0459927937573578, 0.04...   
   prob2  [0.04317666162867696, 0.04379532869590552, 0.0...   
   prob3  [0.04517744601527441, 0.04592570184007105, 0.0...   

                                               Left Forearm  \
0  prob1  [0.016679290744879446, 0.01660171685815603, 0....   
   prob2  [0.011558129380417193, 0.011565868137171273, 0...   
   prob3  [0.02667703873168258, 0.027565982628050708, 0....   
1  prob1  [-0.017788652271901215, -0.01776605348473323, ...   
   prob2  [-0.0209856785581578, -0.02099867753548343, -0...   
...                                                     ...   
48 prob2  [0.011827761401805545, 0.01181138014806391, 0....   
   prob3  [0.012910183532019628, 0.012868782932934019, 0...   
49 prob1  [-0.00758139860262629, -0.007606839995611481, ...   
   prob2  [-0.010865539017050096, -0.010940886133147063,...   
   prob3  [0.029796775021735892, 0.02893595123142797, 0....   

                                                 Left Wrist  \
0  prob1  [0.0150807544700367, 0.015164399314643898, 0.0...   
   prob2  [0.013610073051118712, 0.013646885004090059, 0...   
   prob3  [0.00985274521700916, 0.00998023146164639, 0.0...   
1  prob1  [-0.01